In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from laspy.file import File
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns

import sys

sys.path.insert(0,'..') # So we can import point_density_functions from parent directory
from point_density_functions import *
%load_ext autoreload
%matplotlib notebook

In [2]:
df = pd.read_hdf("../../Data/USGS/las_points_flight_id_18TWK820985.lz")

In [4]:
df.head()

,index,X,Y,Z,intensity,return_number_byte,classification_byte,scan_angle,user_data,pt_src_id,gps_time,x_scaled,y_scaled,z_scaled,latitude,longitude,num_returns,return_num,flight_id
0,1,-205797,6331901,2255,75,9,18,-16,0,41,7.991442e+07,978537.925296,169965.275299,73.982942,40.633190,-74.020580,1,1,0
1,2,-205847,6331907,2258,81,9,18,-16,0,41,7.991442e+07,978536.537554,169965.494221,74.081367,40.633191,-74.020585,1,1,0
2,3,-205897,6331914,2261,79,9,17,-16,0,41,7.991442e+07,978535.149821,169965.749576,74.179792,40.633191,-74.020590,1,1,0
3,4,-205945,6331921,2263,89,9,18,-16,0,41,7.991442e+07,978533.817599,169966.004918,74.245409,40.633192,-74.020595,1,1,0
4,5,-205994,6331927,2264,86,9,18,-16,0,41,7.991442e+07,978532.457613,169966.223834,74.278218,40.633193,-74.020599,1,1,0


In [3]:
def pull_first_scan_gap(wall_face_laefer):
    # Separate return num, only keep the first returns, add scan_gap, sort
    wall_face_laefer['num_returns'] = np.floor(wall_face_laefer['flag_byte']/16).astype(int)
    wall_face_laefer['return_num'] = wall_face_laefer['flag_byte']%16
    first_return_wall = wall_face_laefer[wall_face_laefer['return_num']==1]
    first_return_wall.sort_values(by=['gps_time'],inplace=True)
    first_return_wall.reset_index(inplace=True)
    first_return_wall.loc[1:,'scan_gap'] = [first_return_wall.loc[i+1,'scan_angle'] - first_return_wall.loc[i,'scan_angle'] for i in range(first_return_wall.shape[0]-1)]
    first_return_wall.loc[0,'scan_gap'] = 0
    return first_return_wall

df = pull_first_scan_gap(df)

KeyError: 'flag_byte'

In [ ]:
df[(df['adj_gps_time']>4663807)&(df['adj_gps_time']<4663810)]
# df['adj_gps_time'].describe()

In [ ]:
df['adj_gps_time'] = df['gps_time'] - df['gps_time'].min()
# 60/7550

In [ ]:
df[df['scan_gap']>5000].iloc[50:61].head(25)

In [ ]:
df['gps_time'].mean()

In [ ]:
df.loc[5873474:5874486]['scan_gap'].mean()

In [ ]:
# roughly 1890 points per scan
df.loc[10000:10030]

In [ ]:
secperscan = (60187.351027-60187.201232)/20
print("Seconds per scan: {:2.5f}".format(secperscan))
print("Scans per second: {:2.5f}".format(1/secperscan))

In [ ]:
df[(df['gps_time']>178957000)]

In [ ]:
df.iloc[10834180:10834190]

In [ ]:
df['gps_time'].max()-df['gps_time'].min()